In [2]:
pip install dash

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 8.5 MB/s eta 0:00:0000:0100:01
Note: you may need to restart the kernel to use updated packages.


In [4]:
import pandas as pd
import plotly.express as px
from dash import Dash, dcc, html, Input, Output

# ---- Load and prepare data ----
df = pd.read_csv("pp_median_country.csv")

# Filter to country-level entities
countries = df[df["Type"] == "Country/Area"].copy()

# Ensure Year is integer
countries["Year"] = countries["Year"].astype(int)

# Over-65 percentage (from '65+' column)
countries["over65_pct"] = countries["65+"]

# Keep only 5-year increments
countries_5y = countries[countries["Year"] % 5 == 0].copy()

# List of available 5-year steps
years_5y = sorted(countries_5y["Year"].unique())

# Global min/max for static color scale and y-axis
zmin = countries_5y["over65_pct"].min()
zmax = countries_5y["over65_pct"].max()

# ---- Build Dash app ----
app = Dash(__name__)

app.layout = html.Div(
    style={"fontFamily": "Arial", "margin": "10px"},
    children=[
        html.H2(
            "Global Population Aged 65+ (Projections by Year)",
            style={"textAlign": "center"}
        ),

        # Year slider
        html.Div(
            style={"margin": "20px 40px"},
            children=[
                html.Label("Select Year:", style={"fontWeight": "bold"}),
                dcc.Slider(
                    id="year-slider",
                    min=min(years_5y),
                    max=max(years_5y),
                    step=5,
                    value=years_5y[0],
                    marks={int(y): str(y) for y in years_5y},
                    tooltip={"placement": "bottom", "always_visible": True},
                ),
            ],
        ),

        # Two graphs: choropleth and violin
        html.Div(
            style={"display": "flex", "flexWrap": "wrap"},
            children=[
                html.Div(
                    style={"flex": "1 1 60%", "minWidth": "400px"},
                    children=[
                        dcc.Graph(id="map-65plus")
                    ],
                ),
                html.Div(
                    style={"flex": "1 1 40%", "minWidth": "400px"},
                    children=[
                        dcc.Graph(id="violin-65plus")
                    ],
                ),
            ],
        ),
    ],
)

# ---- Callback to update both figures ----
@app.callback(
    Output("map-65plus", "figure"),
    Output("violin-65plus", "figure"),
    Input("year-slider", "value"),
)
def update_visuals(selected_year):
    # Filter for selected year
    dff = countries_5y[countries_5y["Year"] == selected_year].copy()

    # Choropleth map
    fig_map = px.choropleth(
        dff,
        locations="ISO3 Alpha-code",
        color="over65_pct",
        hover_name="Region, subregion, country or area *",
        color_continuous_scale="YlOrRd",
        title=f"Share of Population Aged 65+ by Country — {selected_year}",
    )

    # Country borders
    fig_map.update_traces(
        marker_line_width=0.2,
        marker_line_color="white"
    )

    # Static legend (colorbar) across all years
    fig_map.update_coloraxes(
        cmin=zmin,
        cmax=zmax,
        colorbar=dict(
            title="Population aged 65+ (%)",
            ticks="outside",
        ),
    )

    fig_map.update_layout(
        margin=dict(l=0, r=0, t=60, b=0),
        title=dict(x=0.5, xanchor="center"),
    )

    # Violin chart: distribution across countries for the selected year
    fig_violin = px.violin(
        dff,
        y="over65_pct",
        box=True,               # show embedded boxplot
        points="all",           # show all points
        hover_data=["Region, subregion, country or area *"],
        title=f"Distribution of 65+ Share Across Countries — {selected_year}",
    )

    # Keep y-axis range static over time
    fig_violin.update_yaxes(
        title="Population aged 65+ (%)",
        range=[zmin, zmax],
    )

    fig_violin.update_layout(
        margin=dict(l=40, r=20, t=60, b=40),
        title=dict(x=0.5, xanchor="center"),
    )
    
    fig_violin.update_traces(marker_color="red", line_color="red")

    return fig_map, fig_violin


if __name__ == "__main__":
    app.run(jupyter_mode="tab")


Dash app running on http://127.0.0.1:8050/


<IPython.core.display.Javascript object>